## Data Clean 

In [47]:
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder


In [48]:
## Data load
data = pd.read_csv('Churn_Modelling.csv')
data.head()

RowNumber  CustomerId   Surname  ...  IsActiveMember EstimatedSalary Exited
0          1    15634602  Hargrave  ...               1       101348.88      1
1          2    15647311      Hill  ...               1       112542.58      0
2          3    15619304      Onio  ...               0       113931.57      1
3          4    15701354      Boni  ...               0        93826.63      0
4          5    15737888  Mitchell  ...               1        79084.10      0

[5 rows x 14 columns]

In [49]:
## Preprocess 

#droping irrelevent
data = data.drop(['RowNumber','CustomerId','Surname'],axis=1) 
data.head()

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0          619    France  Female  ...               1        101348.88       1
1          608     Spain  Female  ...               1        112542.58       0
2          502    France  Female  ...               0        113931.57       1
3          699    France  Female  ...               0         93826.63       0
4          850     Spain  Female  ...               1         79084.10       0

[5 rows x 11 columns]

In [50]:
#encoding categorical to numerical
label_encoder_gender=LabelEncoder()
data['Gender'] = label_encoder_gender.fit_transform(data['Gender'])
data.head()
#female--> 0
#male--> 1

CreditScore Geography  Gender  ...  IsActiveMember  EstimatedSalary  Exited
0          619    France       0  ...               1        101348.88       1
1          608     Spain       0  ...               1        112542.58       0
2          502    France       0  ...               0        113931.57       1
3          699    France       0  ...               0         93826.63       0
4          850     Spain       0  ...               1         79084.10       0

[5 rows x 11 columns]

In [51]:
#geography encoder--one hot encode
onehot_encoder_geo = OneHotEncoder()
geo_encoder=onehot_encoder_geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [52]:
onehot_encoder_geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [53]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=onehot_encoder_geo.get_feature_names_out(['Geography']))
geo_encoded_df

Geography_France  Geography_Germany  Geography_Spain
0                  1.0                0.0              0.0
1                  0.0                0.0              1.0
2                  1.0                0.0              0.0
3                  1.0                0.0              0.0
4                  0.0                0.0              1.0
...                ...                ...              ...
9995               1.0                0.0              0.0
9996               1.0                0.0              0.0
9997               1.0                0.0              0.0
9998               0.0                1.0              0.0
9999               1.0                0.0              0.0

[10000 rows x 3 columns]

In [54]:
#combine OHE columns to main data 
data=pd.concat([data.drop('Geography',axis=1),geo_encoded_df],axis=1)
data

CreditScore  Gender  ...  Geography_Germany  Geography_Spain
0             619       0  ...                0.0              0.0
1             608       0  ...                0.0              1.0
2             502       0  ...                0.0              0.0
3             699       0  ...                0.0              0.0
4             850       0  ...                0.0              1.0
...           ...     ...  ...                ...              ...
9995          771       1  ...                0.0              0.0
9996          516       1  ...                0.0              0.0
9997          709       0  ...                0.0              0.0
9998          772       1  ...                1.0              0.0
9999          792       0  ...                0.0              0.0

[10000 rows x 13 columns]

In [55]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('onehot_encoder_geo.pkl','wb') as file:
    pickle.dump(onehot_encoder_geo,file)

In [56]:
#data spliting 
X = data.drop('Exited',axis=1)
y = data['Exited']

#train test split 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#scale these features
scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)


In [57]:
X_train

array([[ 0.35649971,  0.91324755, -0.6557859 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [-0.20389777,  0.91324755,  0.29493847, ..., -0.99850112,
         1.72572313, -0.57638802],
       [-0.96147213,  0.91324755, -1.41636539, ..., -0.99850112,
        -0.57946723,  1.73494238],
       ...,
       [ 0.86500853, -1.09499335, -0.08535128, ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.15932282,  0.91324755,  0.3900109 , ...,  1.00150113,
        -0.57946723, -0.57638802],
       [ 0.47065475,  0.91324755,  1.15059039, ..., -0.99850112,
         1.72572313, -0.57638802]])

In [58]:
with open('scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

In [59]:
X_train.shape

(8000, 12)

## Training the ANN

In [60]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime

In [61]:
(X_train.shape[1],)

(12,)

In [62]:
## building the ann

model = Sequential([
    Dense(64,activation='relu',input_shape=(X_train.shape[1],)), ## HL1 connected to input layer
    Dense(32,activation='relu'), #HL2
    Dense(1,activation='sigmoid')
])

c:\Users\Hanamanthagouda\Desktop\MissionMidsem\GenAI\course\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [63]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [64]:
import tensorflow
opt = tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss = tensorflow.keras.losses.BinaryCrossentropy()
loss

<LossFunctionWrapper(<function binary_crossentropy at 0x0000023A4DD40A40>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>

In [65]:
#compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [66]:
#Set up the tensorboard

log_dir="logs/fit"+datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir,histogram_freq=1)


In [67]:
# set up early stop 
early_stopping_callback= EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)


In [68]:
## Training 
history = model.fit(
    X_train,y_train,validation_data=(X_test,y_test),epochs=100,
    callbacks=[tensorflow_callback,early_stopping_callback]
)

Epoch 1/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 3s 6ms/step - accuracy: 0.7146 - loss: 0.5457 - val_accuracy: 0.8305 - val_loss: 0.3955
Epoch 2/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8396 - loss: 0.3905 - val_accuracy: 0.8505 - val_loss: 0.3617
Epoch 3/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8554 - loss: 0.3620 - val_accuracy: 0.8580 - val_loss: 0.3476
Epoch 4/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8604 - loss: 0.3379 - val_accuracy: 0.8550 - val_loss: 0.3497
Epoch 5/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.8597 - loss: 0.3382 - val_accuracy: 0.8575 - val_loss: 0.3420
Epoch 6/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8709 - loss: 0.3265 - val_accuracy: 0.8605 - val_loss: 0.3429
Epoch 7/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8601 - loss: 0.3343 - val_accuracy: 0.8580 - val_loss: 0.3433
Epoch 8/100
250/250 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8606 - loss: 0.3279 - val_accu

In [69]:
#save the model
model.save('model.h5')

In [70]:
# Load tensorboard
# %load_ext tensorboard

In [71]:
# %tensorboard --logdir logs/fit20250312-235746/train/